# Two-Qubit Hamiltonians in Pauli-Matrix-Notation

$
\newcommand{\Op}[1]{\boldsymbol{\mathsf{\hat{#1}}}}
\newcommand{\unity}{\mathbb{I}}
\newcommand{\Ket}[1]{\left\vert #1 \right\rangle}
\newcommand{\Bra}[1]{\left\langle #1 \right\vert}
$

In [11]:
import numpy as np
from numpy import kron
import sympy
from IPython.display import Math, Latex, display, display_latex

In [12]:
sympy.init_printing()

In [13]:
def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(np.asarray(M1).conjugate().transpose(), np.asarray(M2))).trace()

In [14]:
def decompose(H, lhs=None, show=False, unity=False):
    """
    Decompose Hermitian 4x4 matrix H into Pauli matrices

    Arguments
    =========

    H  :  numpy 4 x 4 matrix
        Hamiltonian to decompose
    lhs:  string
        Latex string describing H. If given, result will be an equation with
        `lhs` on the left hand side.
    show: boolean
        If `True`, print the decomposition in the context of the IPython
        notebook. Otherwise, a latex string will be returned
    unity: boolean
        If `True`, include terms proportional to unity in the decomposition
    """
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = [r'\unity', r'\Op{\sigma}_x', r'\Op{\sigma}_y', r'\Op{\sigma}_z']
    result = ""
    n_terms = 0
    sign = ''
    for i in range(4):
        for j in range(4):
            if not unity:
                if i == j == 0:
                    continue # skip unity
            if i > 0 and j == 0:
                label = labels[i] + '^{(1)}'
            elif i == 0 and j > 0:
                label = labels[j] + '^{(2)}'
            elif i == 0 and j == 0:
                label = r'\unity'
            else:
                label = labels[i] + '^{(1)}' + labels[j] + r'^{(2)}'
            a_ij = 0.25 * HS(kron(S[i], S[j]), H)
            if a_ij != 0.0:
                if a_ij == 1.0:
                    numstr = ''
                else:
                    numstr = sympy.latex(sympy.nsimplify(a_ij, tolerance=0.001,
                                         rational=True))
                if n_terms > 0:
                    sign = '+'
                if numstr.strip().startswith('-'):
                    sign = ''
                result += r'%s %s %s' % (sign, numstr, label)
                n_terms += 1
    if result == '':
        result = '0'
    if lhs is not None:
        result = lhs + ' = ' + result
    if show:
        display(Math(result))
    else:
        return result

In [15]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            for col in row:
                html.append("<td>{0}</td>".format(col))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

def decompose_ham_terms(ham_terms, unity=False):
    table = ListTable()
    table.append(["Prefactor", "Original Term", "Truncated Term"])
    for (prefactor, term, operator) in ham_terms:
        decomposition = decompose(operator, unity=unity)
        table.append(["$%s$" % prefactor, "$%s$" % term, "$%s$" % decomposition])
    return table

## Transmon Landscape

In [16]:
H = np.matrix([[0,1,0,0],
               [1,0,0,0],
               [0,0,0,0],
               [0,0,0,0]])

In [21]:
display(Latex("$%s$" % decompose(H)))

<IPython.core.display.Latex object>

In [19]:
display_latex(decompose(H))

## Transmon Hamiltonian

The original term is given in terms of annihilation operators, which for a two-level system are

In [6]:
b = np.matrix([[0, 0], [1,0]], dtype=np.complex128) # b = b^\dagger (relabel levels)

Together with the identity

In [7]:
id = np.matrix([[1, 0], [0,1]], dtype=np.complex128)

In [8]:
transmon_ham_terms = [
(r'\omega_1 + \frac{\alpha}{2}', r'\Op{b}_1^\dagger \Op{b}_1',                           kron(b.H*b, id)),
(r'\omega_2 + \frac{\alpha}{2}', r'\Op{b}_2^\dagger \Op{b}_2',                           kron(id, b.H*b)),
(r'-\frac{\alpha}{2}',           r'\Op{b}_1^\dagger \Op{b}_1 \Op{b}_1^\dagger \Op{b}_1', kron(b.H*b*b.H*b, id)),
(r'-\frac{\alpha}{2}',           r'\Op{b}_2^\dagger \Op{b}_2 \Op{b}_2^\dagger \Op{b}_2', kron(id, b.H*b*b.H*b)),
(r'J',                           r'\Op{b}_1^\dagger\Op{b}_2 + \Op{b}_1\Op{b}_2^\dagger', kron(b.H, b) + kron(b, b.H)),
(r'\Omega(t)',                   r'\Op{b}_1 + \Op{b}_1^\dagger',                         kron(b + b.H, id)),
(r'\Omega(t)',                   r'\lambda\Op{b}_2 + \lambda\Op{b}_2^\dagger',           kron(id, b + b.H))
]

In [9]:
decompose_ham_terms(transmon_ham_terms)

Prefactor,Original Term,Truncated Term
$\omega_1 + \frac{\alpha}{2}$,$\Op{b}_1^\dagger \Op{b}_1$,$ \frac{1}{2} \Op{\sigma}_z^{(1)}$
$\omega_2 + \frac{\alpha}{2}$,$\Op{b}_2^\dagger \Op{b}_2$,$ \frac{1}{2} \Op{\sigma}_z^{(2)}$
$-\frac{\alpha}{2}$,$\Op{b}_1^\dagger \Op{b}_1 \Op{b}_1^\dagger \Op{b}_1$,$ \frac{1}{2} \Op{\sigma}_z^{(1)}$
$-\frac{\alpha}{2}$,$\Op{b}_2^\dagger \Op{b}_2 \Op{b}_2^\dagger \Op{b}_2$,$ \frac{1}{2} \Op{\sigma}_z^{(2)}$
$J$,$\Op{b}_1^\dagger\Op{b}_2 + \Op{b}_1\Op{b}_2^\dagger$,$ \frac{1}{2} \Op{\sigma}_x^{(1)}\Op{\sigma}_x^{(2)}+ \frac{1}{2} \Op{\sigma}_y^{(1)}\Op{\sigma}_y^{(2)}$
$\Omega(t)$,$\Op{b}_1 + \Op{b}_1^\dagger$,$ \Op{\sigma}_x^{(1)}$
$\Omega(t)$,$\lambda\Op{b}_2 + \lambda\Op{b}_2^\dagger$,$ \Op{\sigma}_x^{(2)}$


## Charge Qubit Hamiltonian

The Hamiltonian is expressed in terms of projectors and jump operators

In [10]:
P0 = np.matrix([[1, 0], [0,0]]); P1 = np.matrix([[0, 0], [0,1]]);
C01 = np.matrix([[0, 1], [0,0]]); C10 = np.matrix([[0, 0], [1,0]]); 

In [11]:
charge_ham_terms = [
  (r'E_C(0-n_g)^2',        r'\Ket{0}\Bra{0}_1',                    kron(P0, id)),
  (r'E_C(1-n_g)^2',        r'\Ket{1}\Bra{1}_1',                    kron(P1, id)),
  (r'E_C(0-n_g)^2',        r'\Ket{0}\Bra{0}_2',                    kron(id, P0)),
  (r'E_C(1-n_g)^2',        r'\Ket{1}\Bra{1}_2',                    kron(id, P1)),
  (r'-\frac{E_J(t)}{2}',   r'\Ket{0}\Bra{1}_1 + \Ket{1}\Bra{0}_1', kron(C01+C10, id)),
  (r'-\frac{E_J(t)}{2}',   r'\Ket{0}\Bra{1}_2 + \Ket{1}\Bra{0}_2', kron(id, C01+C10)),
  (r'\frac{E_{JJ}(t)}{2}', r'\Ket{01}\Bra{10} + \Ket{10}\Bra{01}', kron(C01,C10) + kron(C10,C01))
]

In [12]:
decompose_ham_terms(charge_ham_terms)

Prefactor,Original Term,Truncated Term
$E_C(0-n_g)^2$,$\Ket{0}\Bra{0}_1$,$ \frac{1}{2} \Op{\sigma}_z^{(1)}$
$E_C(1-n_g)^2$,$\Ket{1}\Bra{1}_1$,$ - \frac{1}{2} \Op{\sigma}_z^{(1)}$
$E_C(0-n_g)^2$,$\Ket{0}\Bra{0}_2$,$ \frac{1}{2} \Op{\sigma}_z^{(2)}$
$E_C(1-n_g)^2$,$\Ket{1}\Bra{1}_2$,$ - \frac{1}{2} \Op{\sigma}_z^{(2)}$
$-\frac{E_J(t)}{2}$,$\Ket{0}\Bra{1}_1 + \Ket{1}\Bra{0}_1$,$ \Op{\sigma}_x^{(1)}$
$-\frac{E_J(t)}{2}$,$\Ket{0}\Bra{1}_2 + \Ket{1}\Bra{0}_2$,$ \Op{\sigma}_x^{(2)}$
$\frac{E_{JJ}(t)}{2}$,$\Ket{01}\Bra{10} + \Ket{10}\Bra{01}$,$ \frac{1}{2} \Op{\sigma}_x^{(1)}\Op{\sigma}_x^{(2)}+ \frac{1}{2} \Op{\sigma}_y^{(1)}\Op{\sigma}_y^{(2)}$


## Two-Qubit Gate Hamiltonians

The Hamiltonians generating the gates can be calculated using the `MatrixLog` function in Mathematica. Then, use `ExportString[%, "MTX"]` to export the result to a Matrix we can parse

In [13]:
def read_mathematica_mtx(mtx_str):
    """
    Return matrix for multiline MTX string.
    """
    from scipy.io import mmread
    from StringIO import StringIO
    a = mmread(StringIO(mtx_str.lstrip()))
    return np.around(a, 4)

#### CNOT

In [27]:
H_CNOT = read_mathematica_mtx("""
%%MatrixMarket matrix array complex symmetric
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -1.5707963267948963E+00   -2.7755575615628899E-16
   1.5707963267948963E+00   -1.6653345369377343E-16
  -1.5707963267948963E+00   -2.7755575615628914E-16
""")
decompose((685.0/538.0)*H_CNOT, lhs=None, show=True, unity=True)

#### CPHASE

In [28]:
H_CPHASE = np.matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,0,0,-1]])
decompose(4*H_CPHASE, lhs=None, show=True, unity=True)

#### DCNOT (iSWAP)

In [29]:
H_DCNOT = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   3.3306690738754696E-16   -3.3306690738754691E-16
  -1.5707963267948963E+00    5.5511151231257852E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -1.5707963267948961E+00   -3.8857805861880479E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
""")
decompose((685.0/538.0)*H_DCNOT, lhs=None, show=True, unity=True)

#### SWAP

In [30]:
H_SWAP = read_mathematica_mtx("""
%%MatrixMarket matrix array complex symmetric
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -1.5707963267948963E+00   -2.7755575615628914E-16
   1.5707963267948963E+00   -1.6653345369377341E-16
   0.0000000000000000E+00    0.0000000000000000E+00
  -1.5707963267948963E+00   -2.7755575615628899E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
""")
decompose((685.0/538.0)*H_SWAP, lhs=None, show=True, unity=True)

#### BGATE

In [31]:
H_BGATE = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
  -8.3266726846886741E-17   -2.9915653287919712E-13
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -3.9269908169875112E-01    1.5959455974508420E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00   -2.9931612743896457E-13
  -1.1780972450961458E+00   -2.2391027933793199E-26
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -1.1780972450961453E+00   -2.2391027933793199E-26
   1.1102230246251565E-16   -2.9931612743896462E-13
   0.0000000000000000E+00    0.0000000000000000E+00
  -3.9269908169875106E-01    6.2450045090408243E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -5.5511151231257827E-17   -2.9917041066700488E-13
""")
decompose((685.0/269.0)*H_BGATE, lhs=None, show=True, unity=True)

#### SQRTSWAP

In [25]:
H_SQRTSWAP = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -7.8539816339744783E-01   -5.5511151231257817E-16
   7.8539816339744783E-01    4.1633363423443444E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   7.8539816339744783E-01   -2.6367796834847463E-16
  -7.8539816339744783E-01   -1.1102230246251560E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
""")
decompose((685.0/269.0)*H_SQRTSWAP, lhs=None, show=True, unity=True)

#### SQRTSWAP*

In [26]:
H_SQRTSWAP2 = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   7.8539816339744783E-01   -5.5511151231257817E-16
  -7.8539816339744783E-01    4.1633363423443444E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -7.8539816339744783E-01   -2.6367796834847463E-16
   7.8539816339744783E-01   -1.1102230246251560E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
""")
decompose((685.0/269.0)*H_SQRTSWAP2, lhs=None, show=True, unity=True)

#### SQRTiSWAP

In [32]:
H_SQRTiSWAP = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    6.4004357369630036E-13
  -7.8539816339744850E-01    2.7755575513291566E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
  -7.8539816339744839E-01    1.9428902920706505E-16
  -1.1102230246251565E-16    6.4004357369630046E-13
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
""")
decompose((685.0/269.0)*H_SQRTiSWAP, lhs=None, show=True, unity=True)

#### MGATE

In [103]:
H_MGATE = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
  -1.1102230246251565E-16    6.4004357369630046E-13
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   7.8539816339744839E-01   -1.9428902920706505E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   3.3306690738754696E-16   -3.3306690738754691E-16
   1.5707963267948963E+00   -5.5511151231257852E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   1.5707963267948961E+00    3.8857805861880479E-16
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   7.8539816339744850E-01   -2.7755575513291566E-17
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    6.4004357369630036E-13
""")
decompose((685.0/269.0)*H_MGATE, lhs=None, show=True, unity=True)

#### RGATE

In [104]:
H_RGATE = read_mathematica_mtx("""
%%MatrixMarket matrix array complex general
%Created by Wolfram Mathematica 9.0 : www.wolfram.com
4 4
   3.9269908169872414E-01   -3.8733599661000540E-13
   0.0000000000000000E+00    0.0000000000000000E+00
   0.0000000000000000E+00    0.0000000000000000E+00
   3.9269908169872403E-01   -3.8708619642946474E-13
  -6.4575636497856742E-18   -3.6444979827435170E-17
  -3.9269908169872403E-01   -3.8713476868679209E-13
   1.1780972450961722E+00   -3.8727354656487023E-13
   8.3092553553308263E-17   -3.1613759429798203E-17
   1.9962442377007358E-17    4.1193509289984208E-18
   1.1780972450961726E+00   -3.8719027983802334E-13
  -3.9269908169872381E-01   -3.8699599080871394E-13
   1.8787574132225411E-17   -8.3362540547744410E-17
   3.9269908169872425E-01   -3.8697517412700222E-13
   8.4402469687650765E-18   -4.8008156213507148E-17
  -2.5320740906105866E-17    1.4402446864055480E-16
   3.9269908169872414E-01   -3.8716252426240771E-13
""")
decompose((685.0/269.0)*H_RGATE, lhs=None, show=True, unity=True)

## Finding the $\sqrt{\text{iSWAP}}$ gates

In [46]:
import QDYN

In [68]:
iSWAP = QDYN.local_invariants.iSWAP
sqrt_iSWAP = QDYN.local_invariants.sqrt_iSWAP

In [67]:
S, V = np.linalg.eig(iSWAP)

In [69]:
SR = np.around(V.dot(np.diag(np.sqrt(S)).dot(V.conjugate().transpose())), 5)

In [71]:
QDYN.io.print_matrix(SR)

{ 1.00E+00,      0.0}(        0,        0)(        0,        0)(        0,        0)
(        0,        0){ 7.07E-01,      0.0}(      0.0, 7.07E-01)(        0,        0)
(        0,        0)(      0.0, 7.07E-01){ 7.07E-01,      0.0}(        0,        0)
(        0,        0)(        0,        0)(        0,        0){ 1.00E+00,      0.0}


In [72]:
QDYN.io.print_matrix(sqrt_iSWAP)

{ 1.00E+00,      0.0}(        0,        0)(        0,        0)(        0,        0)
(        0,        0){ 7.07E-01,      0.0}(      0.0, 7.07E-01)(        0,        0)
(        0,        0)(      0.0, 7.07E-01){ 7.07E-01,      0.0}(        0,        0)
(        0,        0)(        0,        0)(        0,        0){ 1.00E+00,      0.0}


In [100]:
def test_permutation(p):
    SR = V.dot(np.diag(p*np.sqrt(S)).dot(V.conjugate().transpose()))
    print "P = ", p
    print "IS SQUARE ROOT: ", np.around(QDYN.linalg.norm(SR.dot(SR) - iSWAP), 12)
    print "c1, c2, c3 = ", np.around(np.array(QDYN.local_invariants.c1c2c3(SR)), 5)
    print "g1, g2, g3 = ", np.around(np.array(QDYN.local_invariants.g1g2g3(SR)), 5)
    print ""
    return SR

In [102]:
test_permutation([1, 1, 1, 1]);
test_permutation([1, 1, 1, -1]);
test_permutation([1, 1, -1, 1]);
test_permutation([1, 1, -1, -1]);
test_permutation([1, -1, 1, 1]);
test_permutation([1, -1, 1, -1]);
test_permutation([1, -1, -1, 1]);
test_permutation([1, -1, -1, -1]);
test_permutation([-1, 1, 1, 1]);
test_permutation([-1, 1, 1, -1]);
test_permutation([-1, 1, -1, 1]);
test_permutation([-1, 1, -1, -1]);
test_permutation([-1, -1, 1, 1]);
test_permutation([-1, -1, 1, -1]);
test_permutation([-1, -1, -1, 1]);
test_permutation([-1, -1, -1, -1]);

P =  [1, 1, 1, 1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.25  0.25  0.  ]
g1, g2, g3 =  [ 0.25 -0.    1.  ]

P =  [1, 1, 1, -1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.5   0.25  0.25]
g1, g2, g3 =  [-0.25 -0.   -1.  ]

P =  [1, 1, -1, 1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.5   0.25  0.25]
g1, g2, g3 =  [-0.25 -0.   -1.  ]

P =  [1, 1, -1, -1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.25  0.25  0.  ]
g1, g2, g3 =  [ 0.25 -0.    1.  ]

P =  [1, -1, 1, 1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.5   0.25  0.25]
g1, g2, g3 =  [-0.25  0.   -1.  ]

P =  [1, -1, 1, -1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.25  0.25  0.  ]
g1, g2, g3 =  [ 0.25  0.    1.  ]

P =  [1, -1, -1, 1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.25  0.25  0.  ]
g1, g2, g3 =  [ 0.25  0.    1.  ]

P =  [1, -1, -1, -1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.5   0.25  0.25]
g1, g2, g3 =  [-0.25  0.   -1.  ]

P =  [-1, 1, 1, 1]
IS SQUARE ROOT:  0.0
c1, c2, c3 =  [ 0.5   0.25  0.25]
g1, g2, g3 =  [-0.25  0.   -1.  ]

P =  [-1, 1, 1,